In [3]:
!pip install requests
!pip install boto3
!pip install pandas

In [2]:
import requests
import json
import time
import boto3
import pandas as pd

# Initialize the Kinesis client
kinesis_client = boto3.client('kinesis', region_name='us-east-1')
s3_client = boto3.client('s3')


def get_all_crypto_symbols():
    # CoinGecko API endpoint for listing all coins
    api_url = "https://api.coingecko.com/api/v3/coins/list"

    # Make the API request
    response = requests.get(api_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Get the IDs of all cryptocurrencies
        crypto_symbols = [crypto['id'] for crypto in data]

        # Use the first 10 cryptocurrencies as an example; you can adjust the number as needed
        cryptocurrencies = crypto_symbols[:50]
        return cryptocurrencies

    else:
        print(f"Error: {response.status_code}")
        return []

def send_to_kinesis(data):
    # Specify your Kinesis stream name
    stream_name = 'gecko'

    # Send data to Kinesis stream
    response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=json.dumps(data),
        PartitionKey="shardId-000000000000"
    )
    #
    print(f"Record sent to Kinesis. ShardId: {response['ShardId']} SequenceNumber: {response['SequenceNumber']}")
    return response['SequenceNumber']

def get_crypto_prices():
    while True:
        # Get the list of cryptocurrencies
        cryptocurrencies = get_all_crypto_symbols()

        # Convert the list of cryptocurrencies to a comma-separated string
        crypto_string = ",".join(cryptocurrencies)

        # CoinGecko API endpoint for simple price
        api_url = "https://api.coingecko.com/api/v3/simple/price"

        # Parameters for the API request
        params = {
            'ids': crypto_string,
            'vs_currencies': 'usd',  # You can change this to other fiat currencies like 'eur', 'gbp', etc.
        }

        # Make the API request
        response = requests.get(api_url, params=params)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Display the prices
            
            for crypto, values in data.items():
                if "usd" in values:
                    price_data = {"crypto": crypto, "price": values['usd']}
                    print(f"{crypto}: {values['usd']} USD")

                # Send data to Kinesis
                secuence_number = send_to_kinesis(price_data)
            
            csv_file = f"{secuence_number}.csv"
            df=pd.DataFrame.from_records(data)
            df.to_csv(csv_file)
            
            
            # Save to S3
            # Convert JSON to binary data
            s3_client.upload_file( csv_file,Bucket='jorge-bigdatalake', Key=f'raw/csv/{secuence_number}.csv')

        else:
            print(f"Error: {response.status_code}")

        # Wait for 90 seconds before making the next request
        time.sleep(90)

if __name__ == "__main__":
    get_crypto_prices()



01coin: 0.00095085 USD
Record sent to Kinesis. ShardId: shardId-000000000000 SequenceNumber: 49646805381693382128190144889235401226454093835741954050
0chain: 0.168574 USD
Record sent to Kinesis. ShardId: shardId-000000000000 SequenceNumber: 49646805381693382128190144889236610152273708464916660226
0-knowledge-network: 0.00170322 USD
Record sent to Kinesis. ShardId: shardId-000000000000 SequenceNumber: 49646805381693382128190144889237819078093323094091366402
0-mee: 6.448e-05 USD
Record sent to Kinesis. ShardId: shardId-000000000000 SequenceNumber: 49646805381693382128190144889239028003912937723266072578
Record sent to Kinesis. ShardId: shardId-000000000000 SequenceNumber: 49646805381693382128190144889240236929732552352440778754
0x: 0.403663 USD
Record sent to Kinesis. ShardId: shardId-000000000000 SequenceNumber: 49646805381693382128190144889241445855552166981615484930
0x0-ai-ai-smart-contract: 0.118895 USD
Record sent to Kinesis. ShardId: shardId-000000000000 SequenceNumber: 49646805381

KeyboardInterrupt: 